In [9]:
%%javascript
$.getScript('../ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [10]:
from IPython.display import Markdown,display,Math
Markdown('[<< back to chapter content](./content.ipynb)')

[<< back to chapter content](./content.ipynb)

In [11]:
%%HTML
<style>
div.prompt {display:none}
</style>

<div id="toc"></div>

In [12]:
%matplotlib inline
import time
import sys
sys.path.insert(0,'..')
from IPython.display import HTML,Image, SVG, YouTubeVideo
from helpers import show_hide

#Image representation

A digital image is discrete. It means that there is somewhere in the acquisition process, most of the time, sampling that occurs.

A digital image is basically a multidimentionnal array of numbers. Each picture element store a numerical value, with 2D images we speak about **pixels** (from PIcture ELements) and for 3D images we use **voxels** (from VOlume ELements).

The image dimentionnality will depend on:

* the spatial dimentionnality: 1D or profile, 2D image, 3D volume

* the temporal dimension: add one dimension for the time when dealing with **sequences**

* the spectral dimension: the number of spectral values associated with on image element



##Spatial dimensions

The spatial dimension is given, typically, by the grid step of the sensor. A 640x480 CMOS sensor will produce a 640x480 pixels grid.

For a flatbed scanner, one dimension will be given by the number of sensor along the acquisition line, while the second dimension will be given by the spacial repetition of a line acquisition.

One remark concerning these images: the shape of the pixels is not always a square, depending on the sensor geometry and/or the sampling speed (i.e.for the scanners).
One have to pay attention to that, in particular when we will extract measures from images (e.g. distance or surface).




##Temporal dimension



##Spectral dimension


#Information

image entropy

$$\sum_i {p_i \log {1\over p_i}}$$

coocurence matrix

lossy image compression (jpeg/dct)

Huffman encoding

run length encoding

image pyramid (compression)


In [13]:
Math('\sum_i {p_i \log {1\over p_i}}')

<IPython.core.display.Math object>

In [14]:
display(Markdown('#AZERTYU'))

#AZERTYU

#Color representation


    color systems
    multispectral imaging (remote sensing)
    color unmixing



In [15]:
print('Last updated: %s' %time.strftime('%d/%m/%Y'))
HTML(show_hide)

Last updated: 19/07/2015
